# P2 MAAAI 

PREPROCESADO DE DATOS:

In [59]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np

In [60]:


# Cargar CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype('float32') / 255.0  # Normalizar imágenes a [0,1]
x_test = x_test.astype('float32') / 255.0

# Dividir en datos etiquetados y no etiquetados (80% sin etiquetas)
np.random.seed(42)
unlabeled_indices = np.random.choice(len(x_train), size=int(0.8 * len(x_train)), replace=False)
labeled_indices = np.setdiff1d(np.arange(len(x_train)), unlabeled_indices)

x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train[labeled_indices]
x_train_unlabeled = x_train[unlabeled_indices]

print(f"Etiquetadas: {x_train_labeled.shape}, No etiquetadas: {x_train_unlabeled.shape}")

Etiquetadas: (10000, 32, 32, 3), No etiquetadas: (40000, 32, 32, 3)


CREACIÓN DE LA RED NEURONAL CONVOLUCIONAL

In [61]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping


In [62]:

def crear_modelo():
    model = models.Sequential()

    # Bloque convolucional 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Aplanar
    model.add(layers.Flatten())

    # Bloque denso 1
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.6))

    # Capa de salida
    model.add(layers.Dense(100, activation='softmax'))

    # Compilación del modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model



In [63]:

# Crear modelo
model = crear_modelo()

# Callbacks para mejorar el entrenamiento
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
]

# Entrenamiento del modelo
history = model.fit(
    x_train_labeled, y_train_labeled,
    epochs=20,
    batch_size=256,
    validation_data=(x_test, y_test),
    callbacks=callbacks
)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 368ms/step - accuracy: 0.0304 - loss: 5.4930 - val_accuracy: 0.0103 - val_loss: 4.7753
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 332ms/step - accuracy: 0.0864 - loss: 4.3339 - val_accuracy: 0.0183 - val_loss: 5.0993
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 340ms/step - accuracy: 0.1439 - loss: 3.8950 - val_accuracy: 0.0118 - val_loss: 5.5583
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 324ms/step - accuracy: 0.1818 - loss: 3.5745 - val_accuracy: 0.0193 - val_loss: 5.4232
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 321ms/step - accuracy: 0.2306 - loss: 3.2986 - val_accuracy: 0.0224 - val_loss: 5.5510
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 327ms/step - accuracy: 0.2623 - loss: 3.0896 - val_accuracy: 0.0235 - val_loss: 5.5199
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 330ms/step - accuracy: 0.3155 - loss: 2.8418 - val_accuracy: 0.0220 - val_loss: 5.3836
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 324ms/step - accuracy: 0.3501 - loss: 2.6557 - val_accu

In [64]:
# Evaluamos el rendimiento en el conjunto de prueba
train_loss, train_acc = model.evaluate(x_train_labeled, y_train_labeled, verbose=0)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Self-training: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc, test_acc))


313/313 - 3s - 9ms/step - accuracy: 0.2897 - loss: 3.0718
Self-training: Train Accuracy: 0.8694 - Test Accuracy: 0.2897


##EJERCICIO 3 AUTOENCODER:

In [68]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, Input
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.callbacks import EarlyStopping

In [69]:
# ----- Paso 1: Entrenar el Autoencoder -----

def crear_encoder():
    input_img = Input(shape=(32, 32, 3))
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(x)
    x = layers.BatchNormalization()(x)
    encoded = layers.MaxPooling2D((2, 2))(x)

    encoder = models.Model(input_img, encoded, name="encoder")
    return encoder

def crear_decoder(encoded):
    x = layers.UpSampling2D((2, 2))(encoded)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded

def crear_autoencoder():
    input_img = Input(shape=(32, 32, 3))
    encoder = crear_encoder()
    encoded = encoder(input_img)
    decoded = crear_decoder(encoded)
    
    autoencoder = models.Model(input_img, decoded, name="autoencoder")
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder


In [ ]:
autoencoder, encoder = crear_autoencoder()
# Entrenar autoencoder solo con imágenes no etiquetadas

early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

autoencoder.fit(x_train_unlabeled, x_train_unlabeled, epochs=50, batch_size=128, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 130s 430ms/step - loss: 0.0259 - val_loss: 0.0405
Epoch 2/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 98s 347ms/step - loss: 0.0105 - val_loss: 0.0139
Epoch 3/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 97s 342ms/step - loss: 0.0081 - val_loss: 0.0111
Epoch 4/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 98s 348ms/step - loss: 0.0069 - val_loss: 0.0069
Epoch 5/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 98s 347ms/step - loss: 0.0060 - val_loss: 0.0061
Epoch 6/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 104s 368ms/step - loss: 0.0060 - val_loss: 0.0058
Epoch 7/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 101s 358ms/step - loss: 0.0056 - val_loss: 0.0066
Epoch 8/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 101s 359ms/step - loss: 0.0056 - val_loss: 0.0067
Epoch 9/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 97s 345ms/step - loss: 0.0052 - val_loss: 0.0054
Epoch 10/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 97s 342ms/step - loss: 0.0051 - val_loss: 0.0066
Epoch 11/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 97s 345ms/step - loss: 0.0049 - val_loss: 0.0046
Epoch 12/50
282

In [1]:
# ----- Paso 2: Entrenar el Clasificador -----

def crear_clasificador(encoder):
    encoder.trainable = False  # Congelamos el encoder preentrenado
    input_img = encoder.input
    x = encoder.output
    x = layers.Flatten()(x)
    
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.6)(x)
    
    output = layers.Dense(100, activation='softmax')(x)
    
    classifier = models.Model(input_img, output, name="clasificador")
    classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])
    return classifier


In [2]:

classifier = crear_clasificador(encoder)

# Definir Early Stopping para detener si no mejora la validación en 5 épocas seguidas
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Entrenar solo con datos etiquetados
classifier.fit(x_train_labeled, y_train_labeled, epochs=50, batch_size=128, validation_split=0.1, callbacks=[early_stopping])


NameError: name 'encoder' is not defined